<a href="https://colab.research.google.com/github/Potdooshami/planar-domain-wall-network/blob/main/HomologyZ5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import itertools

def get_powerset(s):
  result = []
  for r in range(len(s)+1):
    for combo in itertools.combinations(s, r):
      result.append(''.join(combo))  # Join the tuple into a string
  return result

my_string = "abc"
powerset = get_powerset(my_string)
print(powerset)  # Output: ['', 'a', 'b', 'c', 'ab', 'ac', 'bc', 'abc']

['', 'a', 'b', 'c', 'ab', 'ac', 'bc', 'abc']


In [ ]:
import numpy as np
import json


def kernel(A, tol=1e-5):
    """
    Return a matrix whose column space is the kernel of A.
    The tol parameter is the threshold below which a singular value is considered to be zero.
    Taken from: https://github.com/kb1dds/simplicialHomology/blob/master/simplicialHomology.py
    """
    _, s, vh = np.linalg.svd(A)
    singular = np.zeros(vh.shape[0], dtype=float)
    singular[:s.size] = s
    null_space = np.compress(singular <= tol, vh, axis=0)
    return null_space.T


def cokernel(A, tol=1e-5):
    """
    Return a matrix whose column space is the cokernel of A.
    The tol parameter is the threshold below which a singular value is considered to be zero.
    Taken from: https://github.com/kb1dds/simplicialHomology/blob/master/simplicialHomology.py
    """
    u, s, _ = np.linalg.svd(A)
    singular = np.zeros(u.shape[1], dtype=float)
    singular[:s.size] = s
    return np.compress(singular <= tol, u, axis=1)


def get_faces(lst):
    """Compute all the possible faces by iteratively deleting vertices"""
    return [lst[:i] + lst[i+1:] for i in range(len(lst))]


def get_coeff(simplex, faces):
    """
    If simplex is not in the list of faces, return 0.
    If it is, return index parity.
    """
    if simplex in faces:
        idx = faces.index(simplex)
        return 1 if idx%2==0 else -1
    else:
        return 0


def boundary(complex):
    """
    Given an abstract simplicial complex specified as a list of lists of vertices, return a
    list of boundary operators in matrix form.
    """
    # Get maximal simplex dimension
    ##maxdim = len(max(complex, key=len))
    # Group simplices by (ascending) dimension and sort them lexicographically
    ##simplices = [sorted([spx for spx in complex if len(spx)==i]) for i in range(1,maxdim+1)]
    simplices = chainSort(complex)
    # Iterate over consecutive groups (dim k and k+1)
    bnd = []
    for spx_k, spx_kp1 in zip(simplices, simplices[1:]):
        mtx = []
        for sigma in spx_kp1:
            faces = get_faces(sigma)
            mtx.append([get_coeff(spx, faces) for spx in spx_k])
        bnd.append(np.array(mtx).T)

    return bnd
def chainSort(complex):
  # Get maximal simplex dimension
    maxdim = len(max(complex, key=len))
    # Group simplices by (ascending) dimension and sort them lexicographically
    simplices = [sorted([spx for spx in complex if len(spx)==i]) for i in range(1,maxdim+1)]
    return simplices


def homology(boundary_ops, tol=1e-5):
    """
    Given a list of boundary operators, return a list of matrices whose columns
    span the homology spaces.
    """
    # Insert zero maps
    mm = boundary_ops[-1].shape[1]
    nn = boundary_ops[0].shape[0]

    boundary_ops.insert(0, np.ones(shape=(0, nn)))
    boundary_ops.append(np.ones(shape=(mm, 0)))

    H = []
    for del_k, del_kp1 in zip(boundary_ops, boundary_ops[1:]):
        # Compute a basis for the kernel of the next map
        kappa = kernel(del_k, tol)
        # The chain complex induces a map m from previous space to the kernel of next map
        # Solve d_{k} = kappa \circ m for m
        psi, _, _, _ = np.linalg.lstsq(kappa, del_kp1, rcond=None)
        # The cokernel of m is precisely those elements of the kernel of the next map
        # that are not in the image of m (or d_k for that matter), that's homology
        ksi = cokernel(psi, tol)
        # Express a basis for the homology thought of as a subspace of C_k
        # using composition of maps
        H.append(np.dot(kappa, ksi))

    return H


def betti(H):
    """Compute the dimensions of each homology space output by the homology() function"""
    return [basis.shape[1] for basis in H]

In [ ]:
#X = ["1", "5", "3", "4", "12", "13", "23", "24", "34", "234"]
#X = ["A", "B", "C", "D", "AB", "AC", "BC", "BD", "CD", "BCD"]
X =["1","2","3","4",
    "12","13","14","23","24","34",
    "123","124","134","234",
    "1234"]
X = get_powerset('ㄱㄴㄷㄹㅁ')
bnd = boundary(X)#list of dk
simps = chainSort(X)#list of Ck


In [ ]:
class Z6:
  def __init__(self, value):
    self.value = value % 6
  def __mul__(self, other):
    return Z6(self.value + other.value)
  def __pow__(self,power):
    return Z6(self.value*power)
  def __str__(self):
    return "z"+ str(self.value)
  def __repr__(self):
    return "z"+ str(self.value)
  def __lt__(self, other):
    return self.value < other.value
  def __eq__(self, other):
      #return self.value == other.value
    if isinstance(other, Z6):
      return self.value == other.value
    return False
  def __hash__(self):
    return hash(self.value)

class Z5:
  def __init__(self, value):
    self.value = value % 5
  def __mul__(self, other):
    return Z5(self.value + other.value)
  def __pow__(self,power):
    return Z5(self.value*power)
  def __str__(self):
    return "z"+ str(self.value)
  def __repr__(self):
    return "z"+ str(self.value)
  def __lt__(self, other):
    return self.value < other.value
  def __eq__(self, other):
      #return self.value == other.value
    if isinstance(other, Z5):
      return self.value == other.value
    return False
  def __hash__(self):
    return hash(self.value)


In [ ]:
def sortAlp(str):
  return ''.join(sorted(str))

In [ ]:
a = list(map(Z5,range(5)))
import pandas as pd
pi0 = pd.DataFrame(a,columns=["Z5"])
pi0['C0'] = simps[0]
pi0 = pi0[['C0','Z5']]
pi0

,C0,Z5
0,ㄱ,z0
1,ㄴ,z1
2,ㄷ,z2
3,ㄹ,z3
4,ㅁ,z4


바운더리 함수 데이터프레임화

In [ ]:
bndf = []
for k in  range(len(bnd)):
  bndf.append(pd.DataFrame(bnd[k],index=simps[k],columns=simps[k+1]))

pi1

In [ ]:
nowPow = bndf[0].values
nowG = pi0["Z5"].values
nowG = np.tile(nowG.reshape(-1,1), (1,nowPow.shape[1]))
C1_ = np.prod(nowG**nowPow,axis=0)
pi1 =pd.DataFrame(simps[1],columns=['C1'])
pi1['C1_'] = C1_
pi1['invC1_'] = C1_**-1
pi1

,C1,C1_,invC1_
0,ㄱㄴ,z1,z4
1,ㄱㄷ,z2,z3
2,ㄱㄹ,z3,z2
3,ㄱㅁ,z4,z1
4,ㄴㄷ,z1,z4
5,ㄴㄹ,z2,z3
6,ㄴㅁ,z3,z2
7,ㄷㄹ,z1,z4
8,ㄷㅁ,z2,z3
9,ㄹㅁ,z1,z4


pi2

In [ ]:
pi2 =pd.DataFrame(simps[2],columns=['C2'])
print(pi2)
def sortVort(vort):
  if len(list(set(vort))) == 2:
      vort = sorted(vort)
  elif len(list(set(vort))) == 3:
    mindex = np.nonzero(np.array(vort) == min(vort))[0][0]
    vort = vort[mindex:] + vort[:mindex]
  elif len(list(set(vort))) == 1:
    pass
  else:
    print("error")
  return vort
C2_ = []
invC2_ = []
C2__ = []
C2_bs = []
for iC2 in range(len(simps[2])):
  triNow = pi2['C2'][iC2]
  triEg = [triNow[0:2],triNow[1:3],triNow[2]+triNow[0]]
  vort = []
  for ind in range(3):
    triEg[ind] = sortAlp(triEg[ind])
    orientation = bndf[1][triNow].loc[triEg[ind]]
    if orientation == 1:
      strCol = 'C1_'
    else:
      strCol = 'invC1_'
    vort.append((pi1[strCol].loc[(pi1['C1'] == triEg[ind]).values]).values[0])
    '''
  if len(list(set(vort))) == 2:
    vort = sorted(vort)
  elif len(list(set(vort))) == 3:
    mindex = np.nonzero(np.array(vort) == min(vort))[0][0]
    vort = vort[mindex:] + vort[:mindex]
  elif len(list(set(vort))) == 1:
    pass
  else:
    print("error")
    '''
  vort = sortVort(vort)
  avort = sortVort([vort[2]**-1, vort[1]**-1, vort[0]**-1])
  C2_.append(tuple(vort))
  invC2_.append(tuple(avort))
  C2__.append({tuple(vort),tuple(avort)})
  C2_bs.append(min([tuple(vort),tuple(avort)]))
pi2['C2_']=C2_
pi2['invC2_']=invC2_
pi2['C2_basis']=C2__
pi2['C2_orientation'] = (pi2['C2_']<pi2['invC2_'])*2-1
pi2['C2_bs'] = C2_bs
pi2['C2_bs'].unique()
foo0 = pd.get_dummies(pi2['C2_bs'])
foo1 = ((foo0).astype('int')).values
foo2 = (pi2['C2_orientation']).values
foo2 = np.tile(foo2.reshape(-1,1),(1,2))
pi2L = pd.DataFrame(foo1*foo2,columns=foo0.columns)
pi2[pi2L.columns] = pi2L
pi2L.index = pi2['C2']
pi2L

    C2
0  ㄱㄴㄷ
1  ㄱㄴㄹ
2  ㄱㄴㅁ
3  ㄱㄷㄹ
4  ㄱㄷㅁ
5  ㄱㄹㅁ
6  ㄴㄷㄹ
7  ㄴㄷㅁ
8  ㄴㄹㅁ
9  ㄷㄹㅁ


,"(z1, z1, z3)","(z1, z2, z2)"
C2,,
ㄱㄴㄷ,1,0
ㄱㄴㄹ,0,1
ㄱㄴㅁ,1,0
ㄱㄷㄹ,0,1
ㄱㄷㅁ,0,1
ㄱㄹㅁ,1,0
ㄴㄷㄹ,1,0
ㄴㄷㅁ,0,1
ㄴㄹㅁ,0,1


In [ ]:
#pi3 =pd.DataFrame(simps[3],columns=['C3'])
d3 = bndf[2].values
eqs = pd.DataFrame(np.matmul(pi2L.values.T,d3,),index=pi2L.columns,columns=simps[3])



In [ ]:
def remove_duplicate_equations(arr):
  arr_transposed = np.transpose(arr)
  tuple_rows = [tuple(row) for row in arr_transposed]
  unique_rows = set(tuple_rows)
  return np.array(list(unique_rows)).T

remove_duplicate_equations(eqs.values)

array([[0],
       [0]])

In [ ]:
eqs.values

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [ ]:
U, S, V = np.linalg.svd(eqs.values)
U.shape

(2, 2)

In [ ]:
def is_linearly_independent(matrix, row_vector):
  """
  Checks if a row vector is linearly independent from a matrix.

  Args:
    matrix: The input matrix of shape (m, n).
    row_vector: The row vector of shape (1, n).

  Returns:
    True if the row vector is linearly independent, False otherwise.
  """
  stacked_matrix = np.hstack((matrix, row_vector))
  rank_matrix = np.linalg.matrix_rank(matrix)
  rank_stacked = np.linalg.matrix_rank(stacked_matrix)
  return rank_stacked == rank_matrix + 1
def reduceEq(matrix):
  j = 1

  jHolder = [0]

  stacked_matrix = np.hstack((matrix, matrix[:,jHolder]))

  isFull = False

  while ~isFull:

    isjInd = is_linearly_independent(matrix[:,jHolder],matrix[:,[j]])

    if isjInd:
      jHolder.append(j)
    j = j+1

    isFull = np.linalg.matrix_rank(matrix)==np.linalg.matrix_rank(matrix[:,jHolder])
  return matrix[:,jHolder]

reduceEq(eqs.values)

array([[0],
       [0]])